In [101]:
import pandas as pd

Вводим данные в программу

In [102]:
bookings = pd.read_csv('bookings.csv', sep = ';')

Предобработка столбцов. Все переводим в нижний регистр и убираем пробелы.

In [103]:
bookings.columns = bookings.columns.str.lower().str.replace(' ', '_')
bookings

,hotel,is_canceled,lead_time,arrival_full_date,arrival_date_year,arrival_date_month,arrival_date_week_number,arrival_date_day_of_month,stays_in_weekend_nights,stays_in_week_nights,...,adults,children,babies,meal,country,reserved_room_type,assigned_room_type,customer_type,reservation_status,reservation_status_date
0,Resort Hotel,0,342,2015-07-01,2015,July,27,1,0,0,...,2,0.0,0,BB,PRT,C,C,Transient,Check-Out,2015-07-01
1,Resort Hotel,0,737,2015-07-01,2015,July,27,1,0,0,...,2,0.0,0,BB,PRT,C,C,Transient,Check-Out,2015-07-01
2,Resort Hotel,0,7,2015-07-01,2015,July,27,1,0,1,...,1,0.0,0,BB,GBR,A,C,Transient,Check-Out,2015-07-02
3,Resort Hotel,0,13,2015-07-01,2015,July,27,1,0,1,...,1,0.0,0,BB,GBR,A,A,Transient,Check-Out,2015-07-02
4,Resort Hotel,0,14,2015-07-01,2015,July,27,1,0,2,...,2,0.0,0,BB,GBR,A,A,Transient,Check-Out,2015-07-03
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
119385,City Hotel,0,23,2017-08-30,2017,August,35,30,2,5,...,2,0.0,0,BB,BEL,A,A,Transient,Check-Out,2017-09-06
119386,City Hotel,0,102,2017-08-31,2017,August,35,31,2,5,...,3,0.0,0,BB,FRA,E,E,Transient,Check-Out,2017-09-07
119387,City Hotel,0,34,2017-08-31,2017,August,35,31,2,5,...,2,0.0,0,BB,DEU,D,D,Transient,Check-Out,2017-09-07
119388,City Hotel,0,109,2017-08-31,2017,August,35,31,2,5,...,2,0.0,0,BB,GBR,A,A,Transient,Check-Out,2017-09-07


Перейдем к исследованию данных! Пользователи из каких стран совершили наибольшее число успешных бронирований? 
Бронирование считается успешным, если в дальнейшем не было отменено.

In [110]:
bookings \
    .query("is_canceled == 0") \
    .groupby('country', as_index = False) \
    .aggregate({'is_canceled': 'count'}) \
    .sort_values('is_canceled', ascending=False)

,country,is_canceled
125,PRT,21071
57,GBR,9676
54,FRA,8481
50,ESP,6391
42,DEU,6069
...,...,...
138,SMR,1
101,MLI,1
136,SLE,1
94,MAC,1


На сколько ночей в среднем бронируют отели типа City Hotel? Resort Hotel?

In [114]:
bookings \
    .groupby('hotel', as_index = False) \
    .agg({'stays_total_nights': 'mean'}) \
    .round(2)

,hotel,stays_total_nights
0,City Hotel,2.98
1,Resort Hotel,4.32


Среднее число забронированных ночей в отелях типа city hotel: 2.98.
В resort hotel: 4.32

Иногда тип номера, полученного клиентом, отличается от изначально забронированного. Такое может произойти, например, по причине овербукинга.

Сколько подобных наблюдений встретилось в датасете?

In [122]:
len(bookings.query("assigned_room_type != reserved_room_type"))

14917

На какой месяц чаще всего оформляли бронь в 2016 году? Изменился ли самый популярный месяц в 2017?

In [132]:
bookings.query("arrival_date_year == 2016") \
    .groupby('arrival_date_month', as_index=False) \
    .agg({'hotel': 'count'}) \
    .sort_values('hotel', ascending=False)

,arrival_date_month,hotel
10,October,6203
8,May,5478
0,April,5428
11,September,5394
6,June,5292
1,August,5063
7,March,4824
5,July,4572
9,November,4454
3,February,3891


In [128]:
bookings.query("arrival_date_year == 2017") \
    .groupby('arrival_date_month', as_index=False) \
    .agg({'hotel': 'count'}) \
    .sort_values('hotel', ascending=False)

,arrival_date_month,hotel
7,May,6313
0,April,5661
5,June,5647
4,July,5313
6,March,4970
1,August,4925
2,February,4177
3,January,3681


В 2016 чаще всего бронировали отель на октябрь - 6203 бронирования.
В 2017 году ситуация изменилась, чаще всего стали бронировать в мае - 6313 бронирований.

На какой месяц (arrival_date_month) бронирования отеля типа City Hotel отменялись чаще всего в 2015? 2016? 2017? 

In [156]:
bookings \
    .query("hotel == 'City Hotel' and is_canceled == 1") \
    .groupby(['arrival_date_year', 'arrival_date_month']) \
    .agg({'arrival_date_month': 'count'})

arrival_date_month
arrival_date_year arrival_date_month                    
2015              August                            1232
                  December                           668
                  July                               939
                  November                           301
                  October                           1321
                  September                         1543
2016              April                             1539
                  August                            1247
                  December                          1072
                  February                           930
                  January                            438
                  July                              1043
                  June                              1720
                  March                             1108
                  May                               1436
                  November                          1360
                  October                           1947
                  September                         1567
2017              April                             1926
                  August                            1123
                  February                           971
                  January                           1044
                  July                              1324
                  June                              1808
                  March                             1278
                  May                               2217

В 2015 - Сентябрь. В 2016 - Октябрь. В 2017 - Май.

Какое количество взрослых, детей, младенцов в среднем делают заявку?

In [169]:
bookings[['adults', 'children', 'babies']].mean()

adults      1.856403
children    0.103890
babies      0.007949
dtype: float64

Для какого отеля среднее значения суммы детей и младенцев наибольшее?

In [197]:
bookings['total_kids'] = bookings['children'] + bookings['babies']

bookings.groupby('hotel') \
    .agg({'total_kids': 'mean'}) \
    .round(2)

,total_kids
hotel,
City Hotel,0.10
Resort Hotel,0.14


Для загородного отеля(Resort Hotel) больше - 0.14

Посчитать долю оттока(Churn rate) для клиентов с детьми и без детей. 

In [254]:
round(
    (bookings.query('is_canceled == 1 and total_kids == 0').shape[0]) / (bookings.query('total_kids == 0').shape[0]) * 100,
    2)

37.22

In [255]:
round(
    (bookings.query('is_canceled == 1 and total_kids > 0').shape[0]) / (bookings.query('total_kids > 0').shape[0]) * 100,
    2)

34.92

Доля оттока клиентов без детей больше и равна 37.22%
Доля оттока клиентов с детьми равна 34.92%